In [94]:
# librerie

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql import Row
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.clustering import KMeans

In [2]:
# sessione

sc = SparkContext(appName="DDAM_Project", master="local[*]")
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("DDAM_Project") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
sdf = spark.read.csv("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa04/credit_train.csv", sep=",",
                     inferSchema=True, header=True)

columns = sdf.schema.names

# rinominare le colonne sotituendo lo spazio con l'underscore
for col in columns:
    sdf = sdf.withColumnRenamed(col, col.replace(' ', '_'))

columns = sdf.schema.names

sdf.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Loan_Status: string (nullable = true)
 |-- Current_Loan_Amount: integer (nullable = true)
 |-- Term: string (nullable = true)
 |-- Credit_Score: integer (nullable = true)
 |-- Annual_Income: integer (nullable = true)
 |-- Years_in_current_job: string (nullable = true)
 |-- Home_Ownership: string (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- Monthly_Debt: double (nullable = true)
 |-- Years_of_Credit_History: double (nullable = true)
 |-- Months_since_last_delinquent: string (nullable = true)
 |-- Number_of_Open_Accounts: integer (nullable = true)
 |-- Number_of_Credit_Problems: integer (nullable = true)
 |-- Current_Credit_Balance: integer (nullable = true)
 |-- Maximum_Open_Credit: integer (nullable = true)
 |-- Bankruptcies: string (nullable = true)
 |-- Tax_Liens: string (nullable = true)



In [4]:
rdd = sdf.rdd
rdd.first()

Row(Loan_ID='14dd8831-6af5-400b-83ec-68e61888a048', Customer_ID='981165ec-3274-42f5-a3b4-d104041a9ca9', Loan_Status='Fully Paid', Current_Loan_Amount=445412, Term='Short Term', Credit_Score=709, Annual_Income=1167493, Years_in_current_job='8 years', Home_Ownership='Home Mortgage', Purpose='Home Improvements', Monthly_Debt=5214.74, Years_of_Credit_History=17.2, Months_since_last_delinquent='NA', Number_of_Open_Accounts=6, Number_of_Credit_Problems=1, Current_Credit_Balance=228190, Maximum_Open_Credit=416746, Bankruptcies='1', Tax_Liens='0')

# Attributi con Data Type incoerente

analizziamo gli "attributi problematici": 'Years_in_current_job', 'Months_since_last_delinquent', 'Bankruptcies', 'Tax_Liens'. perché vengono letti con type 'String' da Spark quando invece il loro significato farebbe pensare a valori di tipo numerico.

In [5]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

problematic_columns = ['Years_in_current_job', 'Months_since_last_delinquent', 'Bankruptcies', 'Tax_Liens']

for col in problematic_columns:
    sql = """
    SELECT DISTINCT {0}
    FROM Bank_Loan_Dataset
    ORDER BY {0}
    """.format(col)

    spark.sql(sql).show(150)

+--------------------+
|Years_in_current_job|
+--------------------+
|                null|
|              1 year|
|           10+ years|
|             2 years|
|             3 years|
|             4 years|
|             5 years|
|             6 years|
|             7 years|
|             8 years|
|             9 years|
|            < 1 year|
|                 n/a|
+--------------------+

+----------------------------+
|Months_since_last_delinquent|
+----------------------------+
|                        null|
|                           0|
|                           1|
|                          10|
|                         100|
|                         104|
|                         106|
|                         107|
|                         108|
|                          11|
|                         110|
|                         114|
|                         115|
|                         118|
|                          12|
|                         120|
|                  

In [6]:
# le Row sono tipi particolari di Tuple, quindi sono oggetti immutabili.
# per sostituire i valori dunque trasformiamo le Row in Dictionaries.

rdd = rdd.map(lambda row: row.asDict())

- 'Years_in_current_job'

sono presenti 4222 valori dell'attributo 'Years_in_current_job' uguali a 'n/a'. Pensiamo si possa trattare di soggetti senza lavoro. Il valore è quindi coerente e viene mantenuto così.

In [7]:
rdd.filter(lambda d: d['Years_in_current_job'] == 'n/a').count()

4222

- 'Months_since_last_delinquent'

sono presenti 53141 valori dell'attributo 'Months_since_last_delinquent' uguali a NA. la nostra interpretazione di questo valore è che il soggetto in questione non ha mai commesso nessun reato. quest'interpretazione dervia anche dal fatto che i valori 'NA' siano la maggior parte.

trasformiamo i valori 'NA' in -1, in modo da rendere numerico l'attributo, i cui valori quindi potranno essere confrontati.

In [8]:
rdd.filter(lambda d: d['Months_since_last_delinquent'] == 'NA').count()

53141

In [9]:
def change_value(d):
    if d['Months_since_last_delinquent'] == 'NA':
        d['Months_since_last_delinquent'] = -1
    if d['Months_since_last_delinquent'] is not None:
        d['Months_since_last_delinquent'] = int(d['Months_since_last_delinquent'])
    return d

In [10]:
rdd = rdd.map(change_value)

In [11]:
rdd.filter(lambda d: d['Months_since_last_delinquent'] == -1).count()

53141

- 'Bankruptcies'

sono presenti 204 valori dell'attributo 'Bankruptcies' uguali a NA. Poichè il valore 0 è presente si pensa possa trattarsi di missing values, quindi li trasformiamo in None.

In [12]:
rdd.filter(lambda d: d['Bankruptcies'] == 'NA').count()

204

In [13]:
rdd.filter(lambda d: d['Bankruptcies'] is None).count()

514

In [14]:
def change_value(d):
    if d['Bankruptcies'] == 'NA':
        d['Bankruptcies'] = None
    if d['Bankruptcies'] is not None:
        d['Bankruptcies'] = int(d['Bankruptcies'])
    return d

In [15]:
rdd = rdd.map(change_value)

In [16]:
rdd.filter(lambda d: d['Bankruptcies'] is None).count()

718

- 'Tax_Liens'

sono presenti 10 valori dell'attributo 'Tax_Liens' uguali a NA. Poichè il valore 0 è presente si pensa possa trattarsi di missing values, quindi li trasformiamo in None.

In [17]:
rdd.filter(lambda d: d['Tax_Liens'] == 'NA').count()

10

In [18]:
rdd.filter(lambda d: d['Tax_Liens'] is None).count()

514

In [19]:
def change_value(d):
    if d['Tax_Liens'] == 'NA':
        d['Tax_Liens'] = None
    if d['Tax_Liens'] is not None:
        d['Tax_Liens'] = int(d['Tax_Liens'])
    return d

In [20]:
rdd = rdd.map(change_value)

In [21]:
rdd.filter(lambda d: d['Tax_Liens'] is None).count()

524

In [22]:
# trasformiamo nuovamente l'RDD di Dictionaries in un RDD di Rows

rdd = rdd.map(lambda x: Row(**x))
rdd.first()

Row(Annual_Income=1167493, Bankruptcies=1, Credit_Score=709, Current_Credit_Balance=228190, Current_Loan_Amount=445412, Customer_ID='981165ec-3274-42f5-a3b4-d104041a9ca9', Home_Ownership='Home Mortgage', Loan_ID='14dd8831-6af5-400b-83ec-68e61888a048', Loan_Status='Fully Paid', Maximum_Open_Credit=416746, Monthly_Debt=5214.74, Months_since_last_delinquent=-1, Number_of_Credit_Problems=1, Number_of_Open_Accounts=6, Purpose='Home Improvements', Tax_Liens=0, Term='Short Term', Years_in_current_job='8 years', Years_of_Credit_History=17.2)

In [23]:
sdf = rdd.toDF()
sdf.printSchema()

# gestione degli errori semantici

alcuni attributi hanno dei valori non coerenti con il significato che noi reputiamo possa avere l'attributo (essando sprovvisti di adeguata documentazione su di essi). Alcuni di questi errori sono stati scoperti in fasi più avanzate del progetto (ex Data Understanding).

- il valore dell'attributo 'Current_Credit_Balance' non può essere superiore al valore dell'attributo 'Maximum_Open_Credit'

In [27]:
def semantic_errors(row):
    if row['Current_Credit_Balance'] is not None and row['Maximum_Open_Credit'] is not None:
        return row['Current_Credit_Balance'] < row['Maximum_Open_Credit']
    else:
        return row

In [28]:
rdd = rdd.filter(semantic_errors)
rdd.count()

- ci sono 4526 valori dell'attributo 'Credit_Score' con valore superiore a 4000, che è un valore troppo distante rispetto a quelli generici che assume questo attributo (si aggirano intorno a 200).

In [29]:
def semantic_errors(row):
    if row['Credit_Score'] is not None:
        return row['Credit_Score'] < 4000
    else:
        return row

In [30]:
rdd = rdd.filter(semantic_errors)
rdd.count()

- ci sono valori dell'attributo 'Maximum_Open_Credit' che sono pari a 99.999.999.  un numero eccessivamente più alto rispetto a tutti gli altri valori e che per questo viene eliminato.

In [31]:
def semantic_errors(row):
    if row['Current_Loan_Amount'] is not None:
        return row['Current_Loan_Amount'] != 99999999
    else:
        return row

In [32]:
rdd = rdd.filter(semantic_errors)
rdd.count()

In [34]:
# mximum open credit di 100 000 000
# annual income

In [35]:
# eliminiamo le righe dove entrambi gli attributi 'Loan_ID' e 'Customer_ID' sono nulli,
# che corrispondono alle righe dove tutti i valori di tutti gli attributi sono nulli

rdd = rdd.filter(lambda row: not ( (row['Loan_ID'] is None) and (row['Customer_ID'] is None) ))
rdd.count()

83390

In [36]:
# eliminiamo le righe duplicate

rdd = rdd.distinct()
rdd.count()

73230

In [37]:
sdf = rdd.toDF()

- Problema: ci sono coppie di righe con tutti i valori duplicati eccetto per le due colonne 'Credit_Score' e 'Annual_Income', per le quali uno dei due valori è presente e l'altro è nullo.

Soluzione: si raggruppa per tutti gli attributi tranne quei due e poi si calcola la media di quei due. In questo modo se le uniche due righe uguali sono quelle con un valore nullo e uno non nullo per quegli attributi, lo media sarà uguale al valore non nullo; se invece ci fossero altre righe ugauli ma con altri valori diversi non nulli per quegli attributi, viene effettivamente calcolata la media, il che è auspicabile considerando che tutto il resto della riga è uguale e quindi si tratta molto probabilmente dello stesso oggetto, duplicato per errore, di cui dunque prendiamo un valore medio tra quelli presenti.

In [51]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

columns_temp = [col for col in columns if col != 'Credit_Score' and col != 'Annual_Income']
Project = ''
for col in columns_temp:
    if col == columns[-1]:
        Project += col
        break
    Project += (col + ', ')

sql = """
SELECT {0}, AVG(Credit_Score) AS Credit_Score, AVG(Annual_Income) AS Annual_Income
FROM Bank_Loan_Dataset
GROUP BY {0}
""".format(Project)

sdf = spark.sql(sql)

- come si nota i customers e loans che si ripetevano nel dataset originale erano solo righe duplicate. il dataset pulito non presenta mai valori uguali per questi attributi. possiamo quindi eliminarli.

In [52]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT COUNT(*) AS nbr_rows, COUNT(DISTINCT Customer_ID) AS nbr_customers, COUNT(DISTINCT Loan_ID) AS nbr_loans
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

+--------+-------------+---------+
|nbr_rows|nbr_customers|nbr_loans|
+--------+-------------+---------+
|   69057|        69057|    69057|
+--------+-------------+---------+



In [78]:
columns = [col for col in columns if col != 'Customer_ID' and col != 'Loan_ID']

Project = ''
for col in columns:
    if col == columns[-1]:
        Project += col
        break
    Project += (col + ', ')


sql = """
SELECT {0}
FROM Bank_Loan_Dataset
""".format(Project)

sdf = spark.sql(sql)

sdf

DataFrame[Loan_Status: string, Current_Loan_Amount: bigint, Term: string, Credit_Score: double, Annual_Income: double, Years_in_current_job: string, Home_Ownership: string, Purpose: string, Monthly_Debt: double, Years_of_Credit_History: double, Months_since_last_delinquent: bigint, Number_of_Open_Accounts: bigint, Number_of_Credit_Problems: bigint, Current_Credit_Balance: bigint, Maximum_Open_Credit: bigint, Bankruptcies: bigint, Tax_Liens: bigint]

# Missing Values

In [56]:
def get_nbr_nulls(view_name, columns):
    """funzione per ottenere il numero di valori nulli presenti in ogni colonna dello Spark DF"""

    Project = ''
    for col in columns:
        if col == columns[-1]:
            Project += 'SUM(CASE WHEN {0} IS NULL THEN 1 ELSE 0 END) AS {0}'.format(col)
            break
        Project += 'SUM(CASE WHEN {0} IS NULL THEN 1 ELSE 0 END) AS {0}, '.format(col)


    sql = """\
    SELECT {0}
    FROM {1}\
    """.format(Project, view_name)

    nbr_nulls = spark.sql(sql).collect()[0]

    print('Number of Nulls for each attribute: ')
    for col in columns:
        print(col + ':', '{:>10}'.format(nbr_nulls[col]) )
        
        
    return nbr_nulls

In [79]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

nbr_nulls = get_nbr_nulls(view_name = 'Bank_Loan_Dataset', columns = columns)

Number of Nulls for each attribute: 
Loan_Status:          0
Current_Loan_Amount:          0
Term:          0
Credit_Score:      14838
Annual_Income:      14838
Years_in_current_job:          0
Home_Ownership:          0
Purpose:          0
Monthly_Debt:          0
Years_of_Credit_History:          0
Months_since_last_delinquent:          0
Number_of_Open_Accounts:          0
Number_of_Credit_Problems:          0
Current_Credit_Balance:          0
Maximum_Open_Credit:          1
Bankruptcies:        138
Tax_Liens:          7


Filliamo i Missing Values dividendo il dataset in clusters e usando la Mediana dei valori nei clusters dell'attributo da fillare.

Per effettuare il clustering dobbiamo prima escludere le colonne categoriche e standardizzare il dataset.

In [82]:
columns_categorical = [col.name for col in sdf.schema.fields if isinstance(col.dataType, StringType)]

columns_numerical = [col for col in columns if col not in columns_categorical]

In [95]:
columns_categorical

['Loan_Status', 'Term', 'Years_in_current_job', 'Home_Ownership', 'Purpose']

In [96]:
Project = ''
for col in columns_numerical:
    if col == columns[-1]:
        Project += col
        break
    Project += (col + ', ')


sql = """
SELECT {0}
FROM Bank_Loan_Dataset
""".format(Project)

sdf_numeric = spark.sql(sql)

# **** sei qui

In [99]:
rdd_numeric = sdf_numeric.rdd
rdd_numeric.first()

Row(Current_Loan_Amount=438636, Credit_Score=None, Annual_Income=None, Monthly_Debt=21876.98, Years_of_Credit_History=14.0, Months_since_last_delinquent=-1, Number_of_Open_Accounts=18, Number_of_Credit_Problems=1, Current_Credit_Balance=189601, Maximum_Open_Credit=359898, Bankruptcies=0, Tax_Liens=1)

In [102]:
X = rdd_numeric.map(lambda row: Vectors.dense(row))

In [103]:
X.first()

DenseVector([438636.0, nan, nan, 21876.98, 14.0, -1.0, 18.0, 1.0, 189601.0, 359898.0, 0.0, 1.0])

In [104]:
scaler = StandardScaler(withMean=True, withStd=True)
model = scaler.fit(X)
X_scaled = model.transform(X)

In [107]:
X_scaled.first()

DenseVector([0.7177, nan, nan, 0.2835, -0.6064, -0.7102, 1.3801, 1.7524, -0.2975, nan, nan, nan])

In [108]:
model = KMeans.train(X_scaled, k=10, maxIterations=100, initializationMode="random")

IllegalArgumentException: 'requirement failed'

### Label Encoding of Categorical Attributes

creiamo una nuova struttura dati tabulare dove mappiamo ciascun valore distinto degli attributi caegorici con un numero. Questa sarà utile per sostituire òe stringhe degli attributi categorici 

In [66]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT COUNT(*) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

+--------+
|nbr_rows|
+--------+
|   54116|
+--------+



In [61]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT Loan_Status, COUNT(*) AS nbr_rows
FROM Bank_Loan_Dataset
GROUP BY Loan_Status """

spark.sql(sql).show()

+-----------+--------+
|Loan_Status|nbr_rows|
+-----------+--------+
| Fully Paid|   51088|
|Charged Off|   17969|
+-----------+--------+



In [62]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT Loan_Status, AVG(Credit_Score) AS avg_Credit_Score
FROM Bank_Loan_Dataset
GROUP BY Loan_Status """

spark.sql(sql).show()

+-----------+-----------------+
|Loan_Status| avg_Credit_Score|
+-----------+-----------------+
| Fully Paid|719.9645575699363|
|Charged Off|710.2000954350247|
+-----------+-----------------+



In [86]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT Loan_Status, AVG(Annual_Income) AS avg_Credit_Score
FROM Bank_Loan_Dataset
GROUP BY Loan_Status """

spark.sql(sql).show()

+-----------+------------------+
|Loan_Status|  avg_Credit_Score|
+-----------+------------------+
| Fully Paid|1410691.1453956058|
|Charged Off|1253940.9389215843|
+-----------+------------------+



In [ ]:
pdf = sdf.toPandas()
pdf

# Ricordati!

anche se non esistono valori distinti  COUNT(\*)  può differire da COUNT(DISTINCT \*).

perché il primo conta tutte le righe mentre il secondo conta tutte e sole le righe dove non è presente neanche un NULL value

In [ ]:
sql = """
SELECT COUNT(*) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

In [ ]:
sql = """
SELECT COUNT(Credit_Score) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

In [ ]:
sql = """
SELECT COUNT(DISTINCT *) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()